This code below is not necessary. Its for the categoreies, which we did not need in the end.


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

# --- Paths ---
input_path = Path("/workspaces/bakery_prediction/0_DataPreparation/Processed/03_gi_featured_data.csv")
output_path = Path("/workspaces/bakery_prediction/0_DataPreparation/Processed/04_ls_weather_categorised.csv")

# --- Load CSV ---
df = pd.read_csv(input_path, parse_dates=["Datum"])

# Ensure the column is numeric
df["Wettercode"] = pd.to_numeric(df["Wettercode"], errors="coerce")

# --- Mapping function ---
def map_weather_code(code):
    if np.isnan(code):
        return np.nan
    c = int(code)
    if 0 <= c <= 3:
        return "sunny"
    elif 4 <= c <= 49:
        return "cloudy"
    elif 50 <= c <= 94:
        return "rainy"
    elif 95 <= c <= 99:
        return "thunderstorm"
    else:
        return np.nan

# Replace numeric codes with text labels
df["Wettercode"] = df["Wettercode"].apply(map_weather_code)

# --- Save the new table ---
df.to_csv(output_path, index=False)




' this is the code for the weather variables (1 or 0)

In [3]:
import pandas as pd
from pathlib import Path

# --- Paths ---
input_path = Path("/workspaces/bakery_prediction/0_DataPreparation/Processed/04_ls_weather_categorised.csv")
output_path = Path("/workspaces/bakery_prediction/0_DataPreparation/Processed/05_ls_weather_variables.csv")

# --- Load data ---
df = pd.read_csv(input_path, parse_dates=["Datum"])

# --- Create 0/1 columns manually ---
df["sunny"] = (df["Wettercode"] == "sunny").astype(int)
df["cloudy"] = (df["Wettercode"] == "cloudy").astype(int)
df["rainy"] = (df["Wettercode"] == "rainy").astype(int)
df["thunderstorm"] = (df["Wettercode"] == "thunderstorm").astype(int)

#delete weathercategory column
df = df.drop(columns=["Wettercode"])

# --- Save new dataset ---
df.to_csv(output_path, index=False)







# encode weekdays, months, seasons, warengruppe

In [4]:
import pandas as pd
from pathlib import Path

# --- Paths ---
input_path = Path("/workspaces/bakery_prediction/0_DataPreparation/Processed/05_ls_weather_variables.csv")
output_path = Path("/workspaces/bakery_prediction/0_DataPreparation/Processed/06_ls_encoded_variables.csv")

# --- Load data ---
df = pd.read_csv(input_path, parse_dates=["Datum"])

# ==============================================================
# 1️⃣  WEEKDAY ENCODING (Monday–Sunday)
# ==============================================================
# Ensure "Datum" is datetime
df["Wochentag"] = df["Datum"].dt.day_name()   # e.g. Monday, Tuesday, ...
# Create 0/1 columns for each weekday
df = pd.get_dummies(df, columns=["Wochentag"], prefix="", prefix_sep="")

# ==============================================================
# 2️⃣  MONTH ENCODING (January–December)
# ==============================================================
df["Monat"] = df["Datum"].dt.month_name()
df = pd.get_dummies(df, columns=["Monat"], prefix="", prefix_sep="")

# ==============================================================
# 3️⃣  SEASON ENCODING
# ==============================================================
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Autumn"

df["Season"] = df["Datum"].dt.month.apply(get_season)
df = pd.get_dummies(df, columns=["Season"], prefix="", prefix_sep="")

# ==============================================================
# 4️⃣  WARENGRUPPE (Product group)
# ==============================================================
# Ensure column exists and create 0/1 variables
if "Warengruppe" in df.columns:
    df = pd.get_dummies(df, columns=["Warengruppe"], prefix="Group", prefix_sep="_")

    # ==============================================================
# 🔢 Convert all boolean (True/False) columns to integer (1/0)
# ==============================================================
bool_cols = df.select_dtypes(include="bool").columns
df[bool_cols] = df[bool_cols].astype(int)

# ==============================================================
# 🧩 Reorder columns: weekdays → months → seasons → warengruppe
# ==============================================================
weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
month_order = ["January", "February", "March", "April", "May", "June",
               "July", "August", "September", "October", "November", "December"]
season_order = ["Winter", "Spring", "Summer", "Autumn"]
group_cols = sorted([c for c in df.columns if c.startswith("Group_")])

# Keep all other columns (non-dummy) first
base_cols = [c for c in df.columns if c not in weekday_order + month_order + season_order + group_cols]

# Final ordered list
ordered_cols = base_cols + weekday_order + month_order + season_order + group_cols
df = df[[c for c in ordered_cols if c in df.columns]]  # keep only existing

# add column for temperature regarding seasons
df["Temp_Winter"] = df["Temperatur"] * df["Winter"]
df["Temp_Spring"] = df["Temperatur"] * df["Spring"]
df["Temp_Summer"] = df["Temperatur"] * df["Summer"]
df["Temp_Autumn"] = df["Temperatur"] * df["Autumn"]

# ==============================================================
# 5️⃣  Save the result
# ==============================================================
df.to_csv(output_path, index=False)
print(f"✅ Encoded data saved to:\n{output_path}")


✅ Encoded data saved to:
/workspaces/bakery_prediction/0_DataPreparation/Processed/06_ls_encoded_variables.csv


add rolling averages to df

In [7]:
# read csv
df_roll = pd.read_csv("/workspaces/bakery_prediction/0_DataPreparation/Processed/06_ls_encoded_variables.csv")

# sort via date
df_roll = df_roll.sort_values('Datum')

# calculate Rolling (30 day window)
temp_roll = df_roll['Temperatur'].rolling(30).mean()
wind_roll = df_roll['Windgeschwindigkeit'].rolling(30).mean()
umsatz_roll = df_roll['Umsatz'].rolling(30).mean()

# Spaltenpositionen ermitteln
idx_temp = df_roll.columns.get_loc('Temperatur')
idx_wind = df_roll.columns.get_loc('Windgeschwindigkeit')
idx_ums = df_roll.columns.get_loc('Umsatz')

# Rolling-Spalten einfügen
df_roll.insert(idx_temp, 'Temp_roll', temp_roll)
df_roll.insert(idx_wind + 1, 'Wind_roll', wind_roll)  
df_roll.insert(idx_ums + 2, 'Umsatz_roll', umsatz_roll)

# !!! CSV wieder überschreiben !!!
df_roll.to_csv("/workspaces/bakery_prediction/0_DataPreparation/Processed/06_ls_encoded_variables.csv", index=False)


